## Trabajo Practico


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#### Ejercicio 1

In [2]:
#(a)
df_wine = pd.read_csv('./wine.csv')

In [3]:
#(b)
print(df_wine.columns)
# Variables independientes:
variables_independientes = ['Alcohol', 'Malic_Acid', 'Ash', 'Ash_Alcanity', 'Magnesium',
                            'Total_Phenols', 'Flavanoids', 'Nonflavanoid_Phenols',
                            'Proanthocyanins', 'Color_Intensity', 'Hue', 'OD280', 'Proline']
# Variable dependiente:
variables_dependientes =['Customer_Segment']

Index(['Alcohol', 'Malic_Acid', 'Ash', 'Ash_Alcanity', 'Magnesium',
       'Total_Phenols', 'Flavanoids', 'Nonflavanoid_Phenols',
       'Proanthocyanins', 'Color_Intensity', 'Hue', 'OD280', 'Proline',
       'Customer_Segment'],
      dtype='object')


In [16]:
#(c)
#función que normaliza y centra respecto del promedio 
def normalizacion(df):# recibe un Data Frame que contenga solamente las variables independientes    
    
    # inicialización de variables
    df_norm = pd.DataFrame() 
    
    # Normalización
    for i in df.columns:
        # normalización de cada atributo
        X= df[i] 
        X_prom = X.mean() # calculamos el promedio     # usar pandas o programarlas?
        s = X.std() #calculamos la desviación estándar
        X_norm = [] 
        
        for j in range(len(X)): 
            # normalización de cada elemento
            xj= (X[j] - X_prom)/s
            X_norm.append(xj) 
        
        df_norm[i] = X_norm #agrega cada atributo normalizado al data frame normalizado
    
    return df_norm

df_wine_independientes = df_wine[variables_independientes]

df_wine_norm = normalizacion(df_wine_independientes)
df_wine_dependientes = df_wine[variables_dependientes]

In [5]:
#(d)
#pasamos el data frame a una matriz
matriz_wine_norm = df_wine_norm.values

#calculamos la matriz de covarianza
wine_cov = (matriz_wine_norm.T @ matriz_wine_norm)/177

In [6]:
#(e)
def metodo_potencia(m, its=100):
  
    v = np.random.rand(m.shape[0])  # Inicializar un vector aleatorio
    for i in range(its):
        v = np.dot(m, v)
        v = v / np.linalg.norm(v)  # Normalizar el vector en cada iteración
    
    AutoValor = np.dot(np.dot(m, v), v) / np.dot(v, v) 
    AutoVector = v
    return AutoValor, AutoVector

max_Aval_wine_cov, Avec_de_max_Aval_wine_cov = metodo_potencia(wine_cov)

print(max_Aval_wine_cov, Avec_de_max_Aval_wine_cov)

4.705850252990423 [ 0.1443294  -0.24518758 -0.00205106 -0.23932041  0.14199204  0.39466085
  0.4229343  -0.2985331   0.31342949 -0.0886167   0.29671456  0.37616741
  0.28675223]


In [8]:
#(f)
def todos_Avals_Avects(A, n):
    Avals = []
    Avects = []
    Aval = 0
    Avect = np.zeros(A.shape[0])
    for i in range(0,n):
        A = A - Aval*np.outer(Avect,Avect)
        Aval, Avect = metodo_potencia(A)
        Avals.append(round(Aval, 5))
        Avects.append(Avect)
    return Avals, Avects

A= np.array([[6,-1],[2,3]])   
Avals, V = todos_Avals_Avects(A,2)
    
print(Avals, V)


[5.0, 4.0] [array([0.70710678, 0.70710678]), array([-0.98994949,  0.14142136])]


In [9]:
#(g)
# buscamos autovalores y autovectores de la matriz de covarianza
Avals, Avects = todos_Avals_Avects(wine_cov, 13)

# Sea V  la matriz con los autovectores por columnas  
V = np.array(Avects).T
# Sea D la matriz diagonal con los autovalores en la diagonal ordenados de mayor a menor
D = np.diag(Avals)

# decidimos tomar como los autovalores mas relevantes a los primeros 3

In [18]:
#(g)
#entonces nos quedamos solamente con los primeros 3 componentes principales
Avals, Avects = todos_Avals_Avects(wine_cov, 3)
W = np.array(Avects).T
D = np.diag(Avals)


Y_train = df_wine_dependientes.values
T_train = matriz_wine_norm @ W
#Y_train_T_train = 
# dividimos el Data Frame normalizado en Train y Test Cases

# insertar divicion



def distancia(v1,v2):
    x = 0
    for i in  range(v1.shape[0]):
        x += (v1[i]-v2[i])**2
    return sqrt(x)


def KNN(T_test, T_train, Y_train, K): # devuelve Y_estimado
    cant_CP = T_train.shape[1]
    for i in T_test:
        distancias = []
        for j in T_train:
            distancias.append(distancia(T_test[i],T_train[j]))
            



SyntaxError: expected ':' (919865618.py, line 19)